# Reinforcement Learning and the $N$-Queens Problem
### Viviana Márquez
#### June 27th, 2019

# Introduction

The $n$-queens problem is a classical problem in combinatorics proposed in 1848 by the Max Bezzel, a German chess composer. Since then, the problem has caught the interest of mathematicians and computer scientists alike for its complexity. 

The $n$-queens problem presents the following question:

<center>
    <b>"How many ways can you place $n$-queens in an $n\times n$ chessboard so that no two queens threaten each other?"</b>
</center>

<br>

<center>
    <img src="https://qph.fs.quoracdn.net/main-qimg-04a0f0702f86ecb8931bdd75af9e9a55">
    <i>Queen's possible moves</i>
</center>

Let us recall that a queen in chess can be moved any number of squares vertically, horizontally, or diagonally. As of today, $n=27$ is highest-order number where all the solutions have been enumerated by the [Q27 Project](https://github.com/preusser/q27), although this result still needs to be verified by an independent party. 

<br>

<center>
    <img src="images/sols.gif">
    <i>Known solutions</i>
</center>

It is worth noting that for $n=2$ and $n=3$ there are no possible solutions. For the classical chessboard, $n=8$, the problem is very expensive computationally if approached with brute force. There are $\binom{64}{8} = 4,426,165,368$ possible queen placements, from which only 92 are a solution. For this reason, this problem is of interest for mathematicians and computer scientists.  

<br>

<center>
    <img src="images/queens.png" height='0.1'>
    <i>One of the 92 possible solutions for $n=8$</i>
</center>

# Scope of this project

The $n$-queens problem has many variants, including:

- Finding all solutions for a given $n$.
- Finding *a* solution for a given $n$.
- Benchmarking using the $n$-queens problem. 
- Among others.

For this specific project, we will find *a* solution (queen placement) using Reinforcement Learning.

# The $n$-queens problem as a Reinforcement Learning problem:

<br>
<br>

<big><b>Research question:</b> Can I make a RL agent learn how to find solutions to the n-queen problem?</big>

<br>
<br>

• **Agent:** Machine

• **Environment:** Chessboard

• **State:** Placement of the queens

• **Action:** Coordinates of the next queen to place

• **Reward:** 1 for a valid placement, 0 for a non-valid placement

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
class Queens():
    def __init__(self, n):
        self.n = n
        self.board = np.array([["*" for _ in range(n)] for _ in range(n)])
        self.state = ["*" for _ in range(n)]
        self.available = np.array([["O" for _ in range(n)] for _ in range(n)])
        self.allowed_moves = list(zip(*np.where(self.available == "O")))
    
    def print_board(self):
        """
        Print current state of the game.
        """
        return print(pd.DataFrame(self.board))
    
    def print_available(self):
        """
        Prints available places.
        """
        return print(pd.DataFrame(self.available))
    
    def is_sol(self):
        """
        Check if current state is a solution.
        """
        diagonals = [self.board.diagonal(i) for i in range(-self.n+1,self.n)] + \
                    [self.board[::-1,:].diagonal(i) for i in range(-self.n+1,self.n)]
        for d in diagonals:
            if list(d).count("Q") > 1:
                return False
        if "*" in self.state:
            return False
        return True
    
    def put_queen(self,x,y):
        """
        Place a queen
        """
        if self.available[y][x] == "X": return "Not a valid movement. Try again!"
        self.state[x] = y
        self.board[y][x] = "Q"
        self.update_available(x,y)
        return self.print_board()
    
    def update_available(self,x,y):
        self.available[y][x] = "X"
        self.available[y,:] = "X"
        self.available[:,x] = "X"
        
        aux = np.arange(self.n)
        diag_1 = aux[::-1,None] == aux + self.n - y - x -1
        diag_2 = aux[:,None] == aux + y - x
        self.available[diag_1|diag_2] = "X"
        self.allowed_moves = list(zip(*np.where(self.available == "O")))
          
    def random_board(self):
        """
        Return a random placement of the queens.
        """
        self.restart()
        self.state = np.random.permutation(self.n)
        for i,s in enumerate(self.state):
            self.board[s][i] = "Q"
        self.available[:,:] = "X"
        self.allowed_moves = list(zip(*np.where(self.available == "O")))
        return self.print_board()
    
    def random_movement(self):
        """
        Place a queen at random.
        """
        if len(self.allowed_moves)==0:
            return "No more moves allowed!"
        else:
            x,y = random.choice(self.allowed_moves)
            self.put_queen(y,x)
            
    def sequential_movement(self):
        """
        Place the next available queen.
        """
        if len(self.allowed_moves)==0:
            return "No more moves allowed!"
        else:
            x,y = self.allowed_moves[0]
            self.put_queen(y,x)
            
    def play(self,mode):
        '''
        Given a mode, place queens until there are not more options.
        '''
        self.restart()
        print("*** NEW GAME ***")
        while len(self.allowed_moves)!=0:
            mode()
            print()
        return game.is_sol()
    
    def restart(self):
        """
        Restart the game.
        """
        self.__init__(self.n)
        

# Environment 
Chess board

In [3]:
game = Queens(8)
game.print_board()

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *


# State
Placement of the queens

In [4]:
game.state

['*', '*', '*', '*', '*', '*', '*', '*']

In [5]:
game.random_movement()
game.state

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *


['*', '*', 1, '*', '*', '*', '*', '*']

In [6]:
game.random_movement()
game.state

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  Q  *  *  *  *
7  *  *  *  *  *  *  *  *


['*', '*', 1, 6, '*', '*', '*', '*']

# Action
Coordinates of the next queen to place

In [7]:
game.print_board()

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  Q  *  *  *  *
7  *  *  *  *  *  *  *  *


In [8]:
game.print_available()

   0  1  2  3  4  5  6  7
0  O  X  X  X  O  O  O  O
1  X  X  X  X  X  X  X  X
2  O  X  X  X  O  O  O  X
3  X  O  X  X  X  O  X  O
4  O  X  X  X  O  X  O  O
5  O  O  X  X  X  O  X  O
6  X  X  X  X  X  X  X  X
7  O  O  X  X  X  O  O  O


In [9]:
game.allowed_moves

[(0, 0),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (2, 0),
 (2, 4),
 (2, 5),
 (2, 6),
 (3, 1),
 (3, 5),
 (3, 7),
 (4, 0),
 (4, 4),
 (4, 6),
 (4, 7),
 (5, 0),
 (5, 1),
 (5, 5),
 (5, 7),
 (7, 0),
 (7, 1),
 (7, 5),
 (7, 6),
 (7, 7)]

# Reward
1 [`true`] for a valid placement, 0 [`false`] for a non-valid placement

In [10]:
game.is_sol()

False

# Reinforcement Learning

For this project, two strategies were explored. Random placement and sequential placement (that ultimately resembles a knight movement). We make our agent learn which of the two strategies is the best one to find solutions for the $n$-queens problem.

In [11]:
def strategies(trials=1_000):
    strategies = [game.random_movement, game.sequential_movement] 
    outcomes = []
    for strategy in strategies:
        outcomes.append(sum([game.play(strategy) for _ in range(trials)]))
    return outcomes,strategies

def play_and_learn(n=4, trials=1_000):
    game = Queens(n)
    outcomes,st = strategies(trials)
    result = f"The best strategy is: {str(st[outcomes.index(max(outcomes))])[21:].split()[0]} with {max(outcomes)} points in {trials} trials!"
    return outcomes,st,result

In [12]:
outcomes,strategies,result = play_and_learn()

*** NEW GAME ***
   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  Q  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  Q  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  Q  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  Q  *  *  *  *
4  *  *  *  *  *  *  Q  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  Q  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  Q
3  *  *  *  Q  *  *  *  *
4  *  *  *  *  *  *  Q  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  Q  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  * 

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  Q  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  Q  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  Q
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  Q  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  Q  *  *  *
5  *  *  Q  *  *  *  *  *
6  *  *  *  *  *  *  *  Q
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  Q  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  Q  *
4  *  *  *  *  Q  *  *  *
5  *  *  Q  *  *  *  *  *
6  *  *  *  *  *  *  *  Q
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  Q  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  Q  *
4  *  *  *  *  Q  *  *  *
5  *  *  Q  *  *  *  *  *
6  *  *  *  *  *  *  *  Q
7  *  *  *  *  *  Q  *  *

*** NEW GAME ***
   0  1  2  3  4  5  6  7
0  *  *  *  *  * 

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  Q  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  Q
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  Q  *  *  *  *  *  *
2  *  *  *  *  *  *  Q  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  Q
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  Q  *  *  *  *  *  *
2  *  *  *  *  *  *  Q  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  Q  *  *
5  *  *  *  *  *  *  *  Q
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  Q  *  *  *  *  *  *
2  *  *  *  *  *  *  Q  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  Q  *  *
5  *  *  *  *  *  *  *  Q
6  Q  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  Q 

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  Q  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  Q
6  *  *  *  *  *  *  *  *
7  *  Q  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  Q  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  Q
6  *  *  *  *  *  *  *  *
7  *  Q  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  Q  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  Q
6  *  *  *  Q  *  *  *  *
7  *  Q  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  Q  * 

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  Q  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  Q  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  *  *  Q  *
3  *  *  *  *  *  *  *  *
4  Q  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  Q
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  *  *  Q  *
3  *  *  *  *  *  *  *  *
4  Q  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  *  *  *  *  *  *  *  Q
1  *  * 

...

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  Q  *  *  *  *  *  *
4  *  *  *  Q  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

*** NEW GAME ***
   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  * 

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  Q  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  Q  *  *  *  *  *  *
4  *  *  *  Q  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

*** NEW GAME ***
   0  1  2  3  4  5  6  7
0  Q  *  *  *  * 

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  Q  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  Q  *  *  *  *  *  *
4  *  *  *  Q  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

*** NEW GAME ***
   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  * 

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  Q  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  Q  *  *  *  *  *  *
4  *  *  *  Q  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

*** NEW GAME ***
   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  * 

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  Q  *  *  *  *  *  *
4  *  *  *  Q  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

*** NEW GAME ***
   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  *  *  *  *
1  *  *  Q  *  *  *  *  *
2  *  *  *  *  Q  *  *  *
3  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *

   0  1  2  3  4  5  6  7
0  Q  *  *  *  * 

In [13]:
print(result)

The best strategy is: random_movement with 67 points in 1000 trials!


# Conclusion and Future Directions

In this project, the strategy `random_movement` performed better than the `sequential_movement`. The main challenge for this project was the large state space for the $n$-queens problem where ordinary reinforcement learning is known to perform slower. 

In the state-of-the-art for the $n$-queens problem, it is known that <b>backtracking</b> performs very well since it is a depth first-search algorithm that searches for solutions methodically while satisfying the restrictions on the available space. As a future direction, I would like to explore reinforcement learning with backtracking, as it would help exploring nodes in a smart way, resulting in finding results faster. 

# 📚 References

• Lim, S., Son, K., Park, S., & Lee, S. (2005). The Improvement of Convergence Rate in n-Queen Problem Using Reinforcement learning. Journal of Korean Institute of Intelligent Systems.

• Preußer, T. B., & Engelhardt, M. R. (2016). Putting Queens in Carry Chains, N27. Journal of Signal Processing Systems, 88(2), 185-201. doi:10.1007/s11265-016-1176-8

• Russell, S. J., & Norvig, P. (2018). Artificial intelligence a modern approach. Boston: Pearson.

• Spiering, B. (2019). Reinforcement Learning course @ USF. Retrieved from https://github.com/brianspiering/rl-course